In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import logging
from transformers import pipeline
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
from PipelineCacheWrapper.PipelineCacheWrapper import PipelineCacheWrapper

In [2]:
from TRExData.LamaTRExData import LamaTRExData
from TRExData.AbstractTRExData import AbstractTRExData
from TRExData.HardTRExData import HardKnownTRExData
from SentenceTypologyQueryResults import SentenceTypologyQueryResults
from relation_templates.templates import relations, nominalized_relations, get_templates, get_relation_meta, relations, relation_names, get_relation_cardinality, get_relations_with_last_digit
from ModelHelpers.data_helpers import create_known_filter_getter, create_hard_filter_getter

In [3]:
from TypologyQuerier import TypologyQuerier 
from ResultData.ResultPersister import ResultPersister

In [4]:
#model='bert-base-cased'
#model='bert-base-uncased'
#model='bert-large-cased'
#model='bert-large-uncased'
#model='bert-base-multilingual-cased'
#model='bert-base-multilingual-uncased'
model='roberta-base'
#model='roberta-large'

In [5]:
#data_source = "abstract"
#data_source = "hard and known"
#data_source = "nominalized"
data_source = "complete"

In [6]:
logging.set_verbosity_error()

In [7]:
#MASK = "[MASK]"
MASK = "<mask>"
KEYS = ["active", "passive", "nominalized"]

In [8]:
unmasker = PipelineCacheWrapper('fill-mask', model='bert-base-cased', top_k=10)
def model_to_tokens(sentence):
    result = unmasker(sentence)
    result = sorted(result, key=lambda entry: entry["score"], reverse=True)
    return [entry["token_str"] for entry in result]

In [9]:
get_hard_filter = create_hard_filter_getter(model_to_tokens, get_templates)
get_known_filter = create_known_filter_getter(model_to_tokens, get_templates, get_relation_cardinality)

In [10]:
TREx = LamaTRExData(relations = relations)
#TREx = AbstractTRExData(relations=relations)
#TREx = HardKnownTRExData(get_hard_filter, get_known_filter, relations = relations)

In [11]:
TREx.load()

In [12]:
#relations = ["P19", "P36", "P101", "P103","P106","P108", "P178", "P1001"]
#relations = ["P19", "P413", "P159", "P103"]
#relations = ["P138", "P136"]
#relations = ["P413"]
#relations = nominalized_relations
#relations = ['P37', 'P1412', 'P178', 'P103', 'P36', 'P407', 'P1376', 'P1001', 'P140', 'P463', 'P176'] # considered small
#considered_relations = ['P159', 'P37', 'P1412', 'P17', 'P178', 'P103', 'P36', 'P407', 'P276', 'P31', 'P364', 'P1376', 'P127', 'P279', 'P1001', 'P140', 'P463', 'P176'] #big
#relations = list(TREx.data.keys())
#relations = nominalized_relations
#relations = get_relations_with_last_digit(8)+get_relations_with_last_digit(9)

In [13]:
len(relations)

41

In [14]:
TOP_K = 1

In [15]:
#unmasker = pipeline('fill-mask', model='bert-large-uncased-whole-word-masking', top_k=100)
unmasker = PipelineCacheWrapper('fill-mask', model=model, top_k=10)

In [16]:
sum([len(val) for _, val in TREx.data.items()])

34039

In [17]:
#querier = TypologyQuerier(unmasker, relations, TOP_K, MASK, KEYS)
querier = TypologyQuerier(unmasker, relations, TOP_K, MASK)

In [18]:
querier.query(TREx.data)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 982/982 [02:16<00:00,  7.17it/s]


In [19]:
%%time
unmasker.save_to_cache()

CPU times: user 793 ms, sys: 67.5 ms, total: 860 ms
Wall time: 862 ms


In [20]:
querier.print_global_result()

total Result
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |         0 |   34039 |    0       |
| compound         |      2317 |   34039 |    6.8069  |
| complex          |       826 |   34039 |    2.42663 |
| compound-complex |      2241 |   34039 |    6.58362 |
Total Result for 1:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |         0 |     937 |    0       |
| compound         |         0 |     937 |    0       |
| complex          |         0 |     937 |    0       |
| compound-complex |        28 |     937 |    2.98826 |
Total Result for N:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |         0 |   20928 |    0       |
| compound         |      1163 |   20928 |    5.55715 |
| complex          |       342 |   20928 |    1.6

In [21]:
measure = "k@1 accuracy"
persister = ResultPersister(model=model, measure=measure, data_source=data_source)

In [22]:
querier.results_for_persistence()

[('simple', None, 0.0),
 ('compound', None, 6.806897969975616),
 ('complex', None, 2.4266282793266547),
 ('compound-complex', None, 6.583624665824496),
 ('simple', '1:1', 0.0),
 ('compound', '1:1', 0.0),
 ('complex', '1:1', 0.0),
 ('compound-complex', '1:1', 2.9882604055496262),
 ('simple', 'N:1', 0.0),
 ('compound', 'N:1', 5.557148318042813),
 ('complex', 'N:1', 1.6341743119266054),
 ('compound-complex', 'N:1', 6.1066513761467895),
 ('simple', 'N:M', 0.0),
 ('compound', 'N:M', 9.479218005585674),
 ('complex', 'N:M', 3.975685887957943),
 ('compound-complex', 'N:M', 7.6803022835551165),
 ('simple', 'P159', 0.0),
 ('compound', 'P159', 0.0),
 ('complex', 'P159', 0.0),
 ('compound-complex', 'P159', 0.0),
 ('simple', 'P37', 0.0),
 ('compound', 'P37', 20.28985507246377),
 ('complex', 'P37', 16.666666666666664),
 ('compound-complex', 'P37', 14.285714285714285),
 ('simple', 'P1412', 0.0),
 ('compound', 'P1412', 48.91640866873065),
 ('complex', 'P1412', 0.0),
 ('compound-complex', 'P1412', 51.7

In [23]:
persister.persist(querier.results_for_persistence())
persister.close()

In [24]:
TOP_K = 10

In [25]:
unmasker = PipelineCacheWrapper('fill-mask', model=model, top_k=TOP_K)

In [26]:
querier.reset()
querier.set_top_k(TOP_K)
querier.set_model(unmasker)

In [27]:
querier.query(TREx.data)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 982/982 [00:00<00:00, 32244.43it/s]


In [28]:
%%time
unmasker.save_to_cache()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs


In [29]:
querier.print_global_result()

total Result
| type             |   correct |   total |    accuracy |
|------------------+-----------+---------+-------------|
| simple           |         2 |   34039 |  0.00587561 |
| compound         |      5406 |   34039 | 15.8818     |
| complex          |      3114 |   34039 |  9.14833    |
| compound-complex |      5006 |   34039 | 14.7067     |
Total Result for 1:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |         2 |     937 |   0.213447 |
| compound         |         0 |     937 |   0        |
| complex          |         0 |     937 |   0        |
| compound-complex |        49 |     937 |   5.22946  |
Total Result for N:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |         0 |   20928 |    0       |
| compound         |      3421 |   20928 |   16.3465  |
| complex          |      1908 |   20928 | 

In [30]:
measure = "k@10 accuracy"
persister = ResultPersister(model=model, measure=measure, data_source=data_source)

In [31]:
querier.results_for_persistence()

[('simple', None, 0.005875613267134757),
 ('compound', None, 15.88178266106525),
 ('complex', None, 9.148329856928816),
 ('compound-complex', None, 14.706660007638298),
 ('simple', '1:1', 0.21344717182497333),
 ('compound', '1:1', 0.0),
 ('complex', '1:1', 0.0),
 ('compound-complex', '1:1', 5.229455709711846),
 ('simple', 'N:1', 0.0),
 ('compound', 'N:1', 16.346521406727827),
 ('complex', 'N:1', 9.11697247706422),
 ('compound-complex', 'N:1', 15.261850152905199),
 ('simple', 'N:M', 0.0),
 ('compound', 'N:M', 16.30524067685231),
 ('complex', 'N:M', 9.906357811729915),
 ('compound-complex', 'N:M', 14.481682273698047),
 ('simple', 'P159', 0.0),
 ('compound', 'P159', 0.0),
 ('complex', 'P159', 0.0),
 ('compound-complex', 'P159', 0.0),
 ('simple', 'P37', 0.0),
 ('compound', 'P37', 53.62318840579711),
 ('complex', 'P37', 51.5527950310559),
 ('compound-complex', 'P37', 51.449275362318836),
 ('simple', 'P1412', 0.0),
 ('compound', 'P1412', 71.31062951496389),
 ('complex', 'P1412', 32.301341589

In [32]:
persister.persist(querier.results_for_persistence())
persister.close()

In [33]:
TOP_K = 100

In [34]:
unmasker = PipelineCacheWrapper('fill-mask', model=model, top_k=TOP_K)

In [35]:
querier.reset()
querier.set_top_k(TOP_K)
querier.set_model(unmasker)

In [36]:
querier.query(TREx.data)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 982/982 [02:25<00:00,  6.77it/s]


In [37]:
%%time
unmasker.save_to_cache()

CPU times: user 9.13 s, sys: 495 ms, total: 9.62 s
Wall time: 9.66 s


In [38]:
querier.print_global_result()

total Result
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       429 |   34039 |    1.26032 |
| compound         |      7531 |   34039 |   22.1246  |
| complex          |      5682 |   34039 |   16.6926  |
| compound-complex |      7487 |   34039 |   21.9954  |
Total Result for 1:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |        40 |     937 |    4.26894 |
| compound         |        15 |     937 |    1.60085 |
| complex          |        11 |     937 |    1.17396 |
| compound-complex |        66 |     937 |    7.04376 |
Total Result for N:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       325 |   20928 |    1.55294 |
| compound         |      4920 |   20928 |   23.5092  |
| complex          |      3698 |   20928 |   17.6

In [39]:
measure = "k@100 accuracy"
persister = ResultPersister(model=model, measure=measure, data_source=data_source)

In [40]:
querier.results_for_persistence()

[('simple', None, 1.2603190458004054),
 ('compound', None, 22.12462175739593),
 ('complex', None, 16.692617291929846),
 ('compound-complex', None, 21.995358265518963),
 ('simple', '1:1', 4.268943436499466),
 ('compound', '1:1', 1.6008537886872998),
 ('complex', '1:1', 1.1739594450373532),
 ('compound-complex', '1:1', 7.043756670224119),
 ('simple', 'N:1', 1.5529434250764524),
 ('compound', 'N:1', 23.509174311926607),
 ('complex', 'N:1', 17.670107033639145),
 ('compound-complex', 'N:1', 23.963111620795107),
 ('simple', 'N:M', 0.5257105306390669),
 ('compound', 'N:M', 21.32413339904715),
 ('complex', 'N:M', 16.206669952357483),
 ('compound-complex', 'N:M', 19.76343026121242),
 ('simple', 'P159', 0.0),
 ('compound', 'P159', 1.1375387797311272),
 ('complex', 'P159', 0.2068252326783868),
 ('compound-complex', 'P159', 0.0),
 ('simple', 'P37', 10.766045548654244),
 ('compound', 'P37', 54.865424430641816),
 ('complex', 'P37', 54.865424430641816),
 ('compound-complex', 'P37', 54.761904761904766

In [41]:
persister.persist(querier.results_for_persistence())
persister.close()